In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/selecting-features-using-tree/y_train.csv
/kaggle/input/selecting-features-using-tree/x_train.csv
/kaggle/input/selecting-features-using-tree/__results__.html
/kaggle/input/selecting-features-using-tree/x_submission.csv
/kaggle/input/selecting-features-using-tree/__notebook__.ipynb
/kaggle/input/selecting-features-using-tree/DHSID.csv
/kaggle/input/selecting-features-using-tree/__output__.json
/kaggle/input/selecting-features-using-tree/custom.css
/kaggle/input/maternal-and-child-health-monitoring-in-lmics/training_label.csv
/kaggle/input/maternal-and-child-health-monitoring-in-lmics/sample submission.csv
/kaggle/input/maternal-and-child-health-monitoring-in-lmics/gee_features.csv


In [7]:
from scipy.stats import pearsonr

In [2]:
x_train = pd.read_csv('/kaggle/input/selecting-features-using-tree/x_train.csv')
y_train = pd.read_csv('/kaggle/input/selecting-features-using-tree/y_train.csv')
x_submission = pd.read_csv('/kaggle/input/selecting-features-using-tree/x_submission.csv')
DHSID = pd.read_csv('/kaggle/input/selecting-features-using-tree/DHSID.csv')

In [3]:
from sklearn.model_selection import train_test_split

# Set the random seed for reproducibility
random_seed = 42
np.random.seed(random_seed)

# Splitting the data into train and test sets based on country (80% train, 20% test)
x_train1, x_test1, y_train1, y_test1 = train_test_split(x_train, y_train, test_size=0.2, stratify = x_train["key1"])

# Splitting the train set into train and val sets based on country (75% train, 25% dev)
x_train1, x_val1, y_train1, y_val1 = train_test_split(x_train1, y_train1, test_size=0.25, stratify = x_train1["key1"])

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
from lightgbm import LGBMRegressor

In [5]:
column_names = y_train.columns

In [8]:
# Evaluating the model on the testing and validation sets
for column in column_names:
    # Training an Light GBM model for each target column
    model = LGBMRegressor()
    model.fit(x_train1, y_train1[column])

    # Evaluating the model on the testing set
    y_test_pred = model.predict(x_test1)
    r_test, _ = pearsonr(y_test1[column], y_test_pred)
    pearson_r2_test = r_test ** 2
    print("Squared Pearson r^2 of x_test column {}: {}".format(column, pearson_r2_test))

    # Evaluating the model on the validation set
    y_val_pred = model.predict(x_val1)
    r_val, _ = pearsonr(y_val1[column], y_val_pred)
    pearson_r2_val = r_val ** 2
    print("Squared Pearson r^2 of x_val column {}: {}".format(column, pearson_r2_val))

Squared Pearson r^2 of x_test column Mean_BMI: 0.7147110907935353
Squared Pearson r^2 of x_val column Mean_BMI: 0.7309265628733672
Squared Pearson r^2 of x_test column Median_BMI: 0.6955976749943695
Squared Pearson r^2 of x_val column Median_BMI: 0.7113725779341464
Squared Pearson r^2 of x_test column Unmet_Need_Rate: 0.5972270914958585
Squared Pearson r^2 of x_val column Unmet_Need_Rate: 0.6124600811007748
Squared Pearson r^2 of x_test column Under5_Mortality_Rate: 0.461472513280318
Squared Pearson r^2 of x_val column Under5_Mortality_Rate: 0.47385009577358017
Squared Pearson r^2 of x_test column Skilled_Birth_Attendant_Rate: 0.6089466705507713
Squared Pearson r^2 of x_val column Skilled_Birth_Attendant_Rate: 0.6197083947010582
Squared Pearson r^2 of x_test column Stunted_Rate: 0.39794627482953904
Squared Pearson r^2 of x_val column Stunted_Rate: 0.4150771342629506


Predicting for **submission data**

In [9]:
y_pred = {}
for column in column_names:
    # Training Light GBM Regressor for each target column
    model = LGBMRegressor()
    model.fit(x_train, y_train[column])

    # Evaluate the model on the testing set
    y_pred[column] = model.predict(x_submission)

In [10]:
submission_df3 = pd.DataFrame(y_pred)

In [11]:
# Inserting the 'DHSID' column as the first column in the DataFrame
submission_df3.insert(0, 'DHSID', DHSID)

In [12]:
submission_df3

,DHSID,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
0,AL200800000003,24.824079,24.399313,28.252005,2.381271,74.998360,13.489715
1,AL200800000005,24.847310,24.443412,28.777927,2.582042,74.491277,15.625727
2,AL200800000007,24.944104,24.386320,23.843618,2.308049,71.326547,12.870243
3,AL200800000008,25.236386,24.589875,19.250559,1.864182,75.110558,11.980774
4,AL200800000009,25.073273,24.502993,19.461027,2.069426,73.069575,13.230572
...,...,...,...,...,...,...,...
14995,ZW201500000382,23.571327,23.129249,15.714002,5.541280,71.813905,31.275335
14996,ZW201500000383,24.248724,22.605324,16.561615,5.048558,80.996331,27.615053
14997,ZW201500000386,24.110062,23.477914,21.666533,5.142089,87.511867,29.067732
14998,ZW201500000390,24.795639,24.359351,11.428415,6.116505,91.000135,25.453679


In [13]:
submission_df3.to_csv('submission3.csv',index = False)

**Training by adding some parameters**

In [14]:
# Setting hyperparameters
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 50,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [15]:
# Evaluating the model on the testing and validation sets
for column in column_names:
    # Training an Light GBM model for each target column
    model = LGBMRegressor(**params)
    model.fit(x_train1, y_train1[column])

    # Evaluating the model on the testing set
    y_test_pred = model.predict(x_test1)
    r_test, _ = pearsonr(y_test1[column], y_test_pred)
    pearson_r2_test = r_test ** 2
    print("Squared Pearson r^2 of x_test column {}: {}".format(column, pearson_r2_test))

    # Evaluating the model on the validation set
    y_val_pred = model.predict(x_val1)
    r_val, _ = pearsonr(y_val1[column], y_val_pred)
    pearson_r2_val = r_val ** 2
    print("Squared Pearson r^2 of x_val column {}: {}".format(column, pearson_r2_val))

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 2.621040 seconds.
You can set `force_col_wise=true` to remove the overhead.
Squared Pearson r^2 of x_test column Mean_BMI: 0.7142530582224441
Squared Pearson r^2 of x_val column Mean_BMI: 0.7306551484209947
[Light

Predicting for **submission data**

In [16]:
y_pred = {}
for column in column_names:
    # Training Light GBM Regressor for each target column
    model = LGBMRegressor(**params)
    model.fit(x_train, y_train[column])

    # Evaluate the model on the submission set
    y_pred[column] = model.predict(x_submission)

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 4.674814 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning]

In [17]:
submission_df4 = pd.DataFrame(y_pred)

In [18]:
# Inserting the 'DHSID' column as the first column in the DataFrame
submission_df4.insert(0, 'DHSID', DHSID)

In [19]:
submission_df4

,DHSID,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
0,AL200800000003,25.291797,24.373235,22.749263,3.068085,75.717646,11.344844
1,AL200800000005,25.049159,24.012167,35.516667,3.111162,72.832103,10.953185
2,AL200800000007,25.036430,24.522115,28.675400,2.360750,71.438027,8.507856
3,AL200800000008,25.234908,24.896196,24.277136,1.846389,75.859625,9.000541
4,AL200800000009,25.153108,24.697009,23.530965,1.759695,70.903867,6.998874
...,...,...,...,...,...,...,...
14995,ZW201500000382,23.848425,22.848759,17.028358,5.961709,74.955517,33.114166
14996,ZW201500000383,24.008006,23.109055,12.899273,5.149223,77.479618,27.084875
14997,ZW201500000386,24.485427,23.673493,22.820557,4.831516,92.464502,29.046024
14998,ZW201500000390,24.726619,24.054699,12.294772,6.066305,93.575560,22.640875


In [20]:
submission_df4.to_csv('submission4.csv',index = False)